In [2]:
import torch
import torch.utils.data as utils
from IPython.display import display,Audio
from tqdm.notebook import tqdm
import glob
import numpy as np
import os

import model as mm
import utils as ut
from parameter import Parameter

In [6]:
p=Parameter()
datasets_save_dir = p.datasets_path
split = p.datasets_split #test/val/train
batch_size = p.batch_size
sample_rate = p.sample_rate
num_layer = p.num_layer
model_dir_path = p.model_path

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA is available:", torch.cuda.is_available())

CUDA is available: True


In [4]:
_datasets_path = glob.glob(datasets_save_dir+"/*.npz")
speech_list = []
addnoise_list = []

for file in tqdm(_datasets_path):
    d = np.load(file)    
    speech=torch.from_numpy(d["speech"].astype(np.float32)).clone()
    addnoise=torch.from_numpy(d["addnoise"].astype(np.float32)).clone()
    
    speech_list.append(speech)
    addnoise_list.append(addnoise)
    
num_data = len(speech_list)
a = round(num_data, -2)
if a > num_data:  
    num_usedata = round(num_data-100, -2)
else:
    num_usedata=a
    
tensor_speech = torch.stack(speech_list[:num_usedata])
tensor_addnoise = torch.stack(addnoise_list[:num_usedata])

print("Available data :", num_data)
print("Use data :", num_usedata)

mydataset = utils.TensorDataset(tensor_speech,tensor_addnoise)
data_num = tensor_speech.shape[0]
data_split = [int(data_num * split[0]),
              int(data_num * split[1]),
              int(data_num * split[2])]
test_dataset,val_dataset,train_dataset = utils.random_split(mydataset,data_split)

train_loader = utils.DataLoader(train_dataset,batch_size=batch_size,num_workers=os.cpu_count(),pin_memory=True,shuffle=True)
val_loader = utils.DataLoader(val_dataset,batch_size=batch_size,num_workers=os.cpu_count(),pin_memory=True,shuffle=True)
test_loader = utils.DataLoader(test_dataset,batch_size=batch_size,num_workers=os.cpu_count(),pin_memory=True,shuffle=True)


Available data : 2028
Use data : 2000


In [8]:
model_path = 'model_layer5_20201218_101649.pt'

# model
feat = tensor_addnoise.shape[1]
sequence = tensor_addnoise.shape[2]
model = mm.Net(sequence, feat, num_layer)
model.load_state_dict(torch.load(model_dir_path+model_path))

it = iter(test_loader)
speech,addnoise = next(it)

model.eval()

mask = model(addnoise.float())
h_hat = mask * addnoise

for i in range(h_hat.shape[0]):
    print("Index",i+1)
    addnoiseaudio = ut.tensorstft2audio(addnoise[i,:,:])
    audio = ut.tensorstft2audio(h_hat[i,:,:])
    display(Audio(addnoiseaudio,rate = sample_rate))
    display(Audio(audio,rate = sample_rate))

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable